# Calculate the Fair price of a company stock using DCF method

## Import necessary Libraries

In [568]:
import yfinance as yf
import pandas as pd
import numpy as np

## Company Stock you want to analyse

In [570]:
# Define the stock symbol
symbol = 'NVDA'

# Determine terminal growth rate based on company profile
company_type = "growth"  # Options: 'mature', 'growth', 'conservative'

## Extract Metrics financial metrics

In [572]:
# Create a Ticker object
ticker = yf.Ticker(symbol)

# Fetch the annual financial statements
annual_income_statement = ticker.financials
annual_income_statement = annual_income_statement.loc[:, annual_income_statement.columns.year >= 2021]

annual_cash_flow = ticker.cashflow
annual_cash_flow = annual_cash_flow.loc[:, annual_cash_flow.columns.year >= 2021]

# Convert to DataFrames for easier handling
income_df = annual_income_statement.reset_index()
cash_flow_df = annual_cash_flow.reset_index()

# Try to extract relevant metrics for DCF calculations
metrics = {}
try:
    # Extract dates for each row to use as the first column
    dates = income_df.columns[1:]  # Assuming the first column is 'index', so skip it
    metrics['Date'] = dates  # Add dates as the first column
    
    # Extract metrics from income statement
    if 'Total Revenue' in income_df['index'].values:
        metrics['Total Revenue'] = income_df.set_index('index').loc['Total Revenue'].values
    if 'Total Revenue' in income_df['index'].values:
        metrics['EBIT'] = income_df.set_index('index').loc['EBIT'].values        
    if 'Net Income' in income_df['index'].values:
        metrics['Net Income'] = income_df.set_index('index').loc['Net Income'].values
    if 'Operating Income' in income_df['index'].values:
        metrics['Operating Income'] = income_df.set_index('index').loc['Operating Income'].values
    if 'Interest Expense' in income_df['index'].values:
        metrics['Interest Expense'] = income_df.set_index('index').loc['Interest Expense'].values

    # Extract Capital Expenditures from Cash Flow Statement
    if 'Free Cash Flow' in cash_flow_df['index'].values:
        metrics['Free Cash Flow'] = cash_flow_df.set_index('index').loc['Free Cash Flow'].values
    
    if 'Depreciation And Amortization' in cash_flow_df['index'].values:
        metrics['Depreciation And Amortization'] = cash_flow_df.set_index('index').loc['Depreciation And Amortization'].values

    
    if 'Capital Expenditure' in cash_flow_df['index'].values:
        metrics['Capital Expenditure'] = cash_flow_df.set_index('index').loc['Capital Expenditure'].values

    if 'Change In Working Capital' in cash_flow_df['index'].values:
        metrics['Change In Working Capital'] = cash_flow_df.set_index('index').loc['Change In Working Capital'].values     

    # Convert metrics to a DataFrame for easier viewing
    extracted_metrics = pd.DataFrame(metrics)

    # Display the extracted metrics
    print("\nExtracted Metrics for DCF Calculations:")
    print(extracted_metrics)

except KeyError as e:
    print(f"Error extracting metric: {e}. Please check the available keys in the DataFrame.")



Extracted Metrics for DCF Calculations:
                  Date  Total Revenue           EBIT     Net Income  \
0  2024-01-31 00:00:00  60922000000.0  34075000000.0  29760000000.0   
1  2023-01-31 00:00:00  26974000000.0   4443000000.0   4368000000.0   
2  2022-01-31 00:00:00  26914000000.0  10177000000.0   9752000000.0   
3  2021-01-31 00:00:00  16675000000.0   4593000000.0   4332000000.0   

  Operating Income Interest Expense Free Cash Flow  \
0    32972000000.0      257000000.0  27021000000.0   
1     5577000000.0      262000000.0   3808000000.0   
2    10041000000.0      236000000.0   8132000000.0   
3     4532000000.0      184000000.0   4694000000.0   

  Depreciation And Amortization Capital Expenditure Change In Working Capital  
0                  1508000000.0       -1069000000.0             -3722000000.0  
1                  1544000000.0       -1833000000.0             -2207000000.0  
2                  1174000000.0        -976000000.0             -3363000000.0  
3           

## Calculation for WACC : Weighted Average Cost of Capital
- Represents the average rate a company must pay to finance its assets, reflecting the cost of both debt and equity financing.
- WACC = (E/V × Cost of Equity) + ((D/V × Cost of Debt) * (1 - Tax rate))
- Cost of Equity = Risk-Free Rate + β × (Market Return − Risk-Free Rate)

In [574]:
# Get market cap (equity)
market_cap = ticker.info['marketCap']

# Get debt information (look in the balance sheet)
balance_sheet = ticker.balance_sheet
long_term_debt = balance_sheet.loc['Long Term Debt'][0]
total_debt = long_term_debt


# Calculate total capital
total_capital = market_cap + total_debt

# Assume values for risk-free rate, market return, beta, and tax rate

# Fetch the latest data for the 10-year U.S. Treasury bond (ticker: ^TNX)
treasury_data = yf.Ticker("^TNX").history(period="1d")
risk_free_rate = treasury_data['Close'].iloc[-1] / 100  # Convert to decimal

#risk_free_rate = 0.018
expected_market_return = 0.075

# # Fetch historical data for the S&P 500 index
# snp500 = yf.Ticker("^GSPC")
# data = snp500.history(period="10y")  # Get the last 10 years of data

# # Calculate daily returns and then the annualized return
# data['Daily Return'] = data['Close'].pct_change()
# expected_market_return = (1 + data['Daily Return'].mean()) ** 252 - 1  # 252 trading days in a year

# Check if 'beta' is available in ticker.info
if 'beta' in ticker.info and ticker.info['beta'] is not None:
    beta1 = ticker.info['beta']
    # Beta alternative calculation
    benchmark_symbol = '^GSPC'  # S&P 500 index

    # Fetch historical data for both stock and benchmark
    data_stock = ticker.history(period='1y')
    data_benchmark = yf.Ticker(benchmark_symbol).history(period='1y')

    # Calculate daily returns
    stock_returns = data_stock['Close'].pct_change().dropna()
    benchmark_returns = data_benchmark['Close'].pct_change().dropna()

    # Calculate covariance and variance
    covariance = np.cov(stock_returns, benchmark_returns)[0][1]
    variance = np.var(benchmark_returns)

    # Calculate beta
    beta2 = covariance / variance
    beta = (beta1 + beta2) / 2
    print("Beta from info:", beta)
else:
    # Beta alternative calculation
    benchmark_symbol = '^GSPC'  # S&P 500 index

    # Fetch historical data for both stock and benchmark
    data_stock = ticker.history(period='1y')
    data_benchmark = yf.Ticker(benchmark_symbol).history(period='1y')

    # Calculate daily returns
    stock_returns = data_stock['Close'].pct_change().dropna()
    benchmark_returns = data_benchmark['Close'].pct_change().dropna()

    # Calculate covariance and variance
    covariance = np.cov(stock_returns, benchmark_returns)[0][1]
    variance = np.var(benchmark_returns)

    # Calculate beta
    beta = covariance / variance
    print("Beta calculated from historical data:", beta)

#cost_of_debt = 0.03
interest_expense = annual_income_statement.loc['Interest Expense'].values[0]

# Calculate Cost of Debt
cost_of_debt = interest_expense / total_debt if total_debt else 0


# Extract Income Tax Expense and Pre-Tax Income
income_tax_expense = annual_income_statement.loc['Tax Provision'].values[0]
pre_tax_income = annual_income_statement.loc['Pretax Income'].values[0]

tax_rate = income_tax_expense / pre_tax_income

# Calculate cost of equity
cost_of_equity = risk_free_rate + beta * (expected_market_return - risk_free_rate)

# Calculate WACC
wacc = ((market_cap / total_capital) * cost_of_equity) + ((total_debt / total_capital) * cost_of_debt * (1 - tax_rate))

print(f"WACC: {wacc:.2%}")
print(f"total_debt: {total_debt:,.0f}")
print(f"market_cap: {market_cap:,.0f}")
print(f"total_capital: {total_capital:,.0f}")
print(f"cost_of_debt: {cost_of_debt:,.2%}")
print(f"tax_rate: {tax_rate:,.2%}") 
print(f"cost_of_equity: {cost_of_equity:,.2%}") 
print(f"beta: {beta:,.2f}") 
print(f"Risk-Free Rate: {risk_free_rate:.2%}")
print(f"Expected Market Return: {expected_market_return:.2%}")

Beta from info: 2.217616751716234
WACC: 11.42%
total_debt: 8,459,000,000
market_cap: 3,417,887,735,808
total_capital: 3,426,346,735,808
cost_of_debt: 3.04%
tax_rate: 12.00%
cost_of_equity: 11.44%
beta: 2.22
Risk-Free Rate: 4.27%
Expected Market Return: 7.50%


/var/folders/jk/b919h1bs5jl94wv24kfv4y5m0000gn/T/ipykernel_82543/828601757.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  long_term_debt = balance_sheet.loc['Long Term Debt'][0]
/Users/amit/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/amit/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:782: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/amit/anaconda3/lib/python3.11/site-packages/yfinance/utils.py:782:

## Project FCF using CAGR on FCF

In [576]:
# Drop any rows with NaN values in the 'Free Cash Flow' column to avoid issues
fcf_values = extracted_metrics['Free Cash Flow'].dropna().astype(float).tolist()

# Calculate CAGR
ending_value = fcf_values[0] 
beginning_value = fcf_values[-1] 
n = len(fcf_values) - 1

# CAGR calculation
cagr = (ending_value / beginning_value) ** (1 / n) - 1

# Print CAGR
print("CAGR of FCF:", cagr)

# Output the projected future FCF values
print("Current FCF:",[f"{value:,.0f}" for value in fcf_values])

# Project future FCF using the average growth rate
projected_fcf1 = [fcf_values[0] * (1 + cagr) ** i for i in range(1, 6)]

# Output the projected future FCF values
print("Projected Future FCF:", [f"{value:,.0f}" for value in projected_fcf1])

CAGR of FCF: 0.79219850508893
Current FCF: ['27,021,000,000', '3,808,000,000', '8,132,000,000', '4,694,000,000']
Projected Future FCF: ['48,426,995,806', '86,790,789,489', '155,546,323,179', '278,769,887,873', '499,610,976,309']


## Project FCF using Revenue growth & FCF margin

In [578]:
# Drop any rows with NaN values in the 'Total Revenue' column to avoid issues
revenue_values = extracted_metrics['Total Revenue'].dropna().astype(float).tolist()
print("Revenue values :", [f"{value:,.0f}" for value in revenue_values])

fcf_margins = []

for i in range(0, len(revenue_values)):
    fcf_margin = fcf_values[i]/revenue_values[i]
    fcf_margins.append(fcf_margin)

print("FCF margin:", [f"{value:,.4f}" for value in fcf_margins])
# mean_fcf_margin = sum(fcf_margins) / len(fcf_margins)
# print(mean_fcf_margin)

mean_fcf_margin = max(fcf_margins)
print("Max FCF margin: ",mean_fcf_margin)


Revenue values : ['60,922,000,000', '26,974,000,000', '26,914,000,000', '16,675,000,000']
FCF margin: ['0.4435', '0.1412', '0.3021', '0.2815']
Max FCF margin:  0.4435343554052723


In [579]:
# Revenue growth assumption

# Calculate CAGR
ending_value = revenue_values[0] 
beginning_value = revenue_values[2] 
n = 2

# CAGR calculation
cagr_revenue = (ending_value / beginning_value) ** (1 / n) - 1


print("Revenue growth assumption:", f"{cagr_revenue:,.4f}")

# Project future revenue using the average growth rate
projected_revenue = [revenue_values[0] * (1 + cagr_revenue) ** i for i in range(1, 6)]

# Output the projected future Revenue values
print("Projected Future revenue:", [f"{value:,.0f}" for value in projected_revenue])

# Project future revenue using the average growth rate
projected_fcf2 = [projected_revenue[0] * mean_fcf_margin * (1 + cagr_revenue) ** i for i in range(1, 6)]

# Output the current FCF values
print("Current FCF:", [f"{value:,.0f}" for value in fcf_values])

# Output the projected future Revenue values
print("Projected Future FCFs:", [f"{value:,.0f}" for value in projected_fcf2])

Revenue growth assumption: 0.5045
Projected Future revenue: ['91,658,364,654', '137,901,838,597', '207,476,067,899', '312,151,884,187', '469,638,738,521']
Current FCF: ['27,021,000,000', '3,808,000,000', '8,132,000,000', '4,694,000,000']
Projected Future FCFs: ['61,164,203,091', '92,022,764,038', '138,450,084,741', '208,300,915,163', '313,392,883,355']


In [580]:
# create an average of both of these FCFs methods:

# Calculate the average of both projected FCFs
#projected_fcf = [(f1 + f2) / 2 for f1, f2 in zip(projected_fcf1, projected_fcf2)]

projected_fcf = [((0.4*f1) + (0.6*f2)) for f1, f2 in zip(projected_fcf1, projected_fcf2)]

# Display the average FCFs
print("Average Projected Future FCFs:", [f"{value:,.0f}" for value in projected_fcf])

Average Projected Future FCFs: ['56,069,320,177', '89,929,974,218', '145,288,580,116', '236,488,504,247', '387,880,120,537']


## Get DCFs, Terminal Value, Discounted Terminal Value & Enterprise Value

In [582]:
# Given values
discounted_fcf = [projected_fcf[i] / ((1 + wacc) ** (i + 1)) for i in range(len(projected_fcf))]

# Calculate Terminal Value (using a conservative growth rate)

if company_type == "mature":
    terminal_growth_rate = 0.025  # 2.5% for stable companies
elif company_type == "growth":
    terminal_growth_rate = 0.04   # 4% for high-growth industries
elif company_type == "conservative":
    terminal_growth_rate = 0.01  # 1% conservative for high stability

terminal_value = projected_fcf[-1] * (1 + terminal_growth_rate) / (wacc - terminal_growth_rate)

# Discounted Terminal Value
discounted_terminal_value = terminal_value / ((1 + wacc) ** len(projected_fcf))

# Calculate Enterprise Value
enterprise_value = sum(discounted_fcf) + discounted_terminal_value

# Output results
print("Discounted FCFs:", [f"{value:,.0f}" for value in discounted_fcf])
print("Terminal Value:", f"{terminal_value:,.0f}")
print("Discounted Terminal Value:", f"{discounted_terminal_value:,.0f}")
print("Enterprise Value:", f"{enterprise_value:,.0f}")


Discounted FCFs: ['50,324,236,953', '72,444,978,047', '105,047,858,630', '153,467,941,174', '225,921,239,717']
Terminal Value: 5,439,427,445,285
Discounted Terminal Value: 3,168,201,015,535
Enterprise Value: 3,775,407,270,056


## Calculate Fair Stock Price

In [584]:
# Get the outstanding shares
def get_outstanding_shares(ticker):
    try:
        # Fetch the stock data
        stock = yf.Ticker(ticker)

        # Get Market Capitalization and Current Stock Price
        market_cap = stock.info.get('marketCap')
        current_price = stock.info.get('currentPrice')

        # Calculate Outstanding Shares
        if market_cap is not None and current_price is not None:
            outstanding_shares = market_cap / current_price
            return outstanding_shares
        else:
            return None  # Return None if data is not available
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

outstanding_shares = get_outstanding_shares(symbol)

# Get total debt and cash equivalents for net debt calculation
# total_debt = ticker.info['totalDebt']
# cash_and_equivalents = ticker.info['totalCash']

cash_and_equivalents = balance_sheet.loc['Cash And Cash Equivalents'][0]

# Calculate Net Debt
net_debt = total_debt - cash_and_equivalents

# Print the results
print(f"Outstanding Shares for: {outstanding_shares:,.0f}")
print(f"Total Debt: {total_debt:,.0f}")
print(f"Cash and Cash Equivalents: {cash_and_equivalents:,.0f}")
print(f"Net Debt: {net_debt:,.0f}")
print(f"Equity Value: {enterprise_value - net_debt:,.0f}")

Outstanding Shares for: 24,530,001,334
Total Debt: 8,459,000,000
Cash and Cash Equivalents: 7,280,000,000
Net Debt: 1,179,000,000
Equity Value: 3,774,228,270,056


/var/folders/jk/b919h1bs5jl94wv24kfv4y5m0000gn/T/ipykernel_82543/1240101738.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cash_and_equivalents = balance_sheet.loc['Cash And Cash Equivalents'][0]


In [585]:
# Calculate Fair Stock Price
fair_stock_price = (enterprise_value - net_debt) / outstanding_shares

current_stock_price = ticker.info.get('currentPrice')

# Output the Fair Stock Price
print(f"Fair Stock Price: {fair_stock_price}")
print(f"Current Stock Price: {current_stock_price}")

Fair Stock Price: 153.8617229871957
Current Stock Price: 139.335


In [586]:
# Define a percentage threshold for significant difference
threshold_percentage = 0.10  # 10% difference

# Calculate the percentage difference
percentage_difference = (fair_stock_price - current_stock_price) / current_stock_price

# Logic to evaluate the prices
if percentage_difference > threshold_percentage:
    print("Fair stock price is significantly higher than the current price. Consider buying.")
elif percentage_difference < -threshold_percentage:
    print("Fair stock price is significantly lower than the current price. Consider selling.")
else:
    print("Current stock price is close to fair stock price. Hold.")

Fair stock price is significantly higher than the current price. Consider buying.
